# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train_processado.csv', index_col='ordem')
Y_train = pd.read_csv('y_train_processado.csv', index_col='sujeito')
X_test = pd.read_csv('X_test_processado.csv', index_col='ordem')
Y_test = pd.read_csv('y_test_processado.csv', index_col='sujeito')

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [3]:
# Dividir os dados em treino (75%) e validação (25%)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=42)

# Verificar as dimensões dos dados
print("Dimensões dos dados de treino:", X_train.shape, Y_train.shape)
print("Dimensões dos dados de validação:", X_val.shape, Y_val.shape)
print("Dimensões dos dados de teste:", X_test.shape, Y_test.shape)

Dimensões dos dados de treino: (5514, 562) (5514, 2)
Dimensões dos dados de validação: (1838, 562) (1838, 2)
Dimensões dos dados de teste: (2947, 562) (2947, 2)


### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [4]:
# Rodar uma árvore com profundidade máxima igual a 4
clf = DecisionTreeClassifier(max_depth=4)
clf.fit(X_train, Y_train)

# Obter a importância das variáveis
importances = clf.feature_importances_

# Criar um DataFrame para facilitar a manipulação
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

# Ordenar as variáveis por importância em ordem decrescente
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Obter as três variáveis mais importantes
top_3_variables = importance_df.head(3)

# Imprimir as três variáveis mais importantes
print('Top 3 variáveis mais importantes:')
top_3_variables

Top 3 variáveis mais importantes:


,Feature,Importance
53,53 tGravityAcc-min()-X,0.284479
87,87 tBodyAccJerk-mad()-X,0.253387
560,"560 angle(Y,gravityMean)",0.164249


### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [5]:
X_train_cut = X_train[['57 tGravityAcc-energy()-X', '272 fBodyAcc-mad()-X', '560 angle(Y,gravityMean)']]
X_valid_cut = X_val[['57 tGravityAcc-energy()-X', '272 fBodyAcc-mad()-X', '560 angle(Y,gravityMean)']]
X_test_cut = X_test[['57 tGravityAcc-energy()-X', '272 fBodyAcc-mad()-X', '560 angle(Y,gravityMean)']]

In [6]:
caminho = DecisionTreeClassifier(random_state=2360873).cost_complexity_pruning_path(X_train_cut, Y_train)
ccp_alphas, impurities = caminho.ccp_alphas, caminho.impurities

In [7]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=ccp_alpha).fit(X_train_cut, Y_train)
    clfs.append(clf)

KeyboardInterrupt: 

In [ ]:
train_scores = [clf.score(X_train_cut, Y_train) for clf in clfs]
valid_scores = [clf.score(X_valid_cut, Y_valid) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("Alpha")
ax.set_ylabel("Acurácia")
ax.set_title("Acurácia x Alpha do conjunto de dados de Treino e Validação")
ax.plot(ccp_alphas, train_scores, marker='o', label="Validação",
        drawstyle="steps-post")
ax.plot(ccp_alphas, valid_scores, marker='o', label="Teste",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
ind_melhor_arvore = len(valid_scores) - valid_scores[::-1].index(max(valid_scores)) - 1
melhor_arvore = clfs[ind_melhor_arvore]

print(f'Acurácia da melhor árvore na base de validação: {valid_scores[ind_melhor_arvore]*100:.2f}')

melhor_arvore

### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [ ]:
acuracia_teste = melhor_arvore.score(X_test_cut, y_test)

print(f'Acurácia: {acuracia_teste*100:.2f}')
ConfusionMatrixDisplay.from_estimator(melhor_arvore, X_test_cut, y_test)